# MODNet on the joined PBE and experimental datasets

Here we joined the PBE and experimental datasets, and trained the MODNet model on it. We weighted the data from the experimental dataset a bit more such that: {'pbe_gap':0.3, 'exp_gap':0.7}.

In [1]:
from modnet.preprocessing import MODData
from modnet.models import MODNetModel
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

In [2]:
import logging
logging.getLogger().setLevel(logging.INFO)
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle
import tensorflow.keras.backend as K
from tensorflow.keras.models import model_from_json
from modnet.preprocessing import MODData
from modnet.model_presets import *

class MODNetModel:
    
    def __init__(self,targets,weights,num_neurons=[[64],[32],[16],[16]], n_feat=300,act='relu',out_act='linear'):

        self.targets = targets
        self.n_feat = n_feat
        self.weights = weights
        self.out_act = out_act

        num_layers = [len(x) for x in num_neurons]

        f_temp = [x for subl in targets for x in subl]
        self.targets_flatten = [x for subl in f_temp for x in subl]
        
        if len(self.targets_flatten) > 1:
            self.PP = True
        else:
            self.PP = False
    

        #Build first common block
        f_input = Input(shape=(n_feat,))
        previous_layer = f_input
        for i in range(num_layers[0]):
            previous_layer = Dense(num_neurons[0][i],activation=act)(previous_layer)
            if self.PP:
                previous_layer = BatchNormalization()(previous_layer)
        common_out = previous_layer

        # build intermediate representations
        intermediate_models_out = []
        for i in range(len(targets)):
            previous_layer = common_out
            for j in range(num_layers[1]):
                previous_layer = Dense(num_neurons[1][j],activation=act)(previous_layer)
                if self.PP:
                    previous_layer = BatchNormalization()(previous_layer)
            intermediate_models_out.append(previous_layer)

        #Build outputs
        final_out = []
        for group_idx,group in enumerate(targets):
            for prop_idx in range(len(group)):
                previous_layer = intermediate_models_out[group_idx]
                for k in range(num_layers[2]):
                    previous_layer = Dense(num_neurons[2][k],activation=act)(previous_layer)
                    if self.PP:
                        previous_layer = BatchNormalization()(previous_layer)
                clayer = previous_layer
                temps = []
                for pi in range(len(group[prop_idx])):
                    previous_layer = clayer
                    for li in range(num_layers[3]):
                        previous_layer = Dense(num_neurons[3][li])(previous_layer)
                    out = Dense(1,activation=self.out_act,name=group[prop_idx][pi])(previous_layer)
                    final_out.append(out)

        self.model = tf.keras.models.Model(inputs=f_input, outputs=final_out)

    def fit(self,data:MODData, val_fraction = 0.0, val_key = None, lr=0.001, epochs = 200, batch_size = 128, xscale='minmax', loss='mse', callbacks=None, verbose=1):
        
        if self.n_feat > len(data.get_optimal_descriptors()):
            raise RuntimeError("The model requires more features than computed in data. Please reduce n_feat below or equal to {}".format(len(data.get_optimal_descriptors())))
        self.xscale = xscale
        self.target_names = data.names
        self.optimal_descriptors = data.get_optimal_descriptors()
        x = data.get_featurized_df()[self.optimal_descriptors[:self.n_feat]].values
        #print(x.shape)
        y = data.get_target_df()[self.targets_flatten].values.transpose()
        #print(y.shape)
        
        #Scale the input features:
        if self.xscale == 'minmax':
            self.xmin = x.min(axis=0)
            self.xmax = x.max(axis=0)
            x=(x-self.xmin)/(self.xmax-self.xmin) - 0.5
                
        elif self.xscale == 'standard':
            self.scaler = StandardScaler()
            x = self.scaler.fit_transform(x)

        x = np.nan_to_num(x)
        
        if verbose and self.PP:
            if val_fraction>0:
                print_callback = LambdaCallback(
                  on_epoch_end=lambda epoch,logs: print("epoch {}: loss: {:.3f}, val_loss:{:.3f} val_{}:{:.3f}".format(epoch,logs['loss'],logs['val_loss'],val_key,logs['val_{}_mae'.format(val_key)])))
                verbose = 0
            else:
                print_callback = LambdaCallback(
                    on_epoch_end=lambda epoch,logs: print("epoch {}: loss: {:.3f}".format(epoch,logs['loss'])))
                verbose = 0
            if callbacks is None:
                callbacks = [print_callback]
            else:
                callbacks += [print_callback]

        fit_params = {
            'x': x,
            'y': list(y),
            'epochs': epochs,
            'batch_size': batch_size,
            'verbose': verbose,
            'validation_split': val_fraction,
            'callbacks': callbacks
        }

        #print('compile',flush=True)
        self.model.compile(loss = loss, optimizer=tf.keras.optimizers.Adam(lr=lr), metrics=['mae'], loss_weights=self.weights)
        #print('fit',flush=True)
        
        history = self.model.fit(**fit_params)

        return history


    #def fit_preset(self, data:MODData,verbose=0):
        """
        Chooses an optimal hyper-parametered MODNet model from different presets .
        The data is first fitted on several well working MODNet presets with a validation set (20% of the furnished data).
        The best validating preset is then fitted again on the whole data, and the current model is updated accordingly.
        Args:
            data: MODData object contain training and validation samples.

        Returns: None, object is updated to fit the data.

        """
    #    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=verbose, mode="auto", min_delta=0)
    #    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=verbose, mode="auto", baseline=None,
    #                       restore_best_weights=True)
    #    callbacks = [rlr,es]
    #    val_losses = np.empty((len(MODNET_PRESETS),))
    #    for i,params in enumerate(MODNET_PRESETS):
    #        logging.info("Training preset #{}/{}".format(i+1,len(MODNET_PRESETS)))
    #        n_feat = min(len(data.get_optimal_descriptors()),params['n_feat'])
    #        self.model = MODNetModel(self.targets,self.weights,num_neurons=params['num_neurons'],n_feat=n_feat, act=params['act'],out_act=self.out_act).model
    #        self.n_feat = n_feat
    #        hist = self.fit(data, val_fraction=0.2, lr=params['lr'], epochs=params['epochs'], batch_size=params['batch_size'], loss=params['loss'], callbacks=callbacks, verbose=verbose)
    #        val_loss = np.array(hist.history['val_loss'])[-20:].mean()
    #        val_losses[i] = val_loss
    #        logging.info("Validation loss: {:.3f}".format(val_loss))
    #    best_preset = val_losses.argmin()
    #    logging.info("Preset #{} resulted in lowest validation loss.\nFitting all data...".format(best_preset+1))
    #    n_feat = min(len(data.get_optimal_descriptors()), MODNET_PRESETS[best_preset]['n_feat'])
    #    self.model = MODNetModel(self.targets, self.weights, num_neurons=MODNET_PRESETS[best_preset]['num_neurons'], n_feat=n_feat,
    #                             act=MODNET_PRESETS[best_preset]['act'],out_act=self.out_act).model
    #    self.n_feat = n_feat
    #    self.fit(data, val_fraction=0, lr=MODNET_PRESETS[best_preset]['lr'], epochs=MODNET_PRESETS[best_preset]['epochs'],
    #                    batch_size=MODNET_PRESETS[best_preset]['batch_size'], loss=MODNET_PRESETS[best_preset]['loss'], callbacks=callbacks, verbose=verbose)


    def predict(self,data):
        
        x = data.get_featurized_df()[self.optimal_descriptors[:self.n_feat]].values
        
        #Scale the input features:
        if self.xscale == 'minmax':
            x=(x-self.xmin)/(self.xmax-self.xmin) - 0.5
                
        elif self.xscale == 'standard':
            x = self.scaler.transform(x)
        
        x = np.nan_to_num(x)
        
        if self.PP:
            p = np.array(self.model.predict(x))[:,:,0].transpose()
        else:
            p = np.array(self.model.predict(x))[:,0].transpose()
        predictions = pd.DataFrame(p)
        predictions.columns = self.targets_flatten
        predictions.index = data.structure_ids
        
        return predictions
    
    
    def save(self,filename):
        model = self.model
        self.model = None
        model_json = model.to_json()
        fp = open('{}.json'.format(filename), 'w')
        fp.write(model_json)
        fp.close()
        model.save_weights('{}.h5'.format(filename))
        fp = open('{}.pkl'.format(filename),'wb')
        pickle.dump(self,fp)
        fp.close()
        self.model = model
        print('Saved model')
    
    @staticmethod
    def load(filename):
        fp = open('{}.pkl'.format(filename),'rb')
        mod = pickle.load(fp)
        fp.close()
        fp = open('{}.json'.format(filename), 'r')
        model_json = fp.read()
        fp.close()
        mod.model = model_from_json(model_json)
        mod.model.load_weights('{}.h5'.format(filename))
        return mod

In [2]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [3]:
md_exp = MODData.load('exp_gap_all_mpid')
md_exp.df_targets.columns = ['exp_gap','mp_id']
md_pbe = MODData.load('pbe_gap.zip')
#md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
#md_hse.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7455435c10> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7455435d60> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7455435910> object, created with modnet version 0.1.8~develop
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7455435f70> object, created with modnet version <=0.1.7


In [4]:
# joint exp and pbe

'''

df_exp = md_exp.df_targets

icsds_n =  [int(x.split('-')[1]) for x in list(df_exp.index)]

from pymatgen.ext.matproj import MPRester

with MPRester(api_key='DaZc4G3gfZrVogm6') as mpr:
    res1 = mpr.query({"icsd_ids": {"$in": icsds_n}}, properties=["material_id","icsd_ids"],chunk_size=2000)
    
mapping = {}

for el in res1:
    for icsd in el['icsd_ids']:
        mapping['icsd-'+str(icsd)] = el['material_id']
        

df_exp['mp_id'] = df_exp.index.map(mapping)

md_exp.df_targets = df_exp
#md_exp.save('moddatas/exp_gap_all_mpid')
'''

'''
md_joint = copy.deepcopy(md_exp)
intersec = set(md_pbe.df_targets.index).intersection(set(md_exp.df_targets['mp_id']))
md_joint.df_targets = md_exp.df_targets.set_index('mp_id').join(md_pbe.df_targets,how='inner').loc[intersec]
md_joint.df_featurized = md_pbe.df_featurized.loc[intersec]
md_joint.df_structure = md_pbe.df_structure.loc[intersec]

md_joint.save('moddatas/exp_pbe_joint')
'''

"\nmd_joint = copy.deepcopy(md_exp)\nintersec = set(md_pbe.df_targets.index).intersection(set(md_exp.df_targets['mp_id']))\nmd_joint.df_targets = md_exp.df_targets.set_index('mp_id').join(md_pbe.df_targets,how='inner').loc[intersec]\nmd_joint.df_featurized = md_pbe.df_featurized.loc[intersec]\nmd_joint.df_structure = md_pbe.df_structure.loc[intersec]\n\nmd_joint.save('moddatas/exp_pbe_joint')\n"

In [7]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'train_jointA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1)
        train.save(fpath)
    
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    #phase 1
    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'pbe_gap':0.3, 'exp_gap':0.7},act='elu')
    model.fit_preset(train,verbose=0)
    
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    print(error.head())
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error['exp_gap'].values).mean()
    print('mae')
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7416aec6d0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.424
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.397
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 

           exp_gap   pbe_gap
mp-8046   0.061767  0.025327
mp-12569  2.260568  0.026822
mp-1211  -1.646768  0.025427
mp-1723   0.061406  0.025311
mp-21149  0.054345  0.028356
mae
0.4550654079870071


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7334633430> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.341
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.308
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.334
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.356
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.309
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.337
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.384
INFO:root:Training preset #8/1

            exp_gap   pbe_gap
mp-574273  0.034456  0.017452
mp-20905  -0.540270  0.531685
mp-28911   3.299709  1.852704
mp-22811   0.922594  0.012722
mp-1863    0.978782  0.630005
mae
0.3928549249316345


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f73345ce8b0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.323
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.336
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.338
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.369
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.302
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.312
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.319
INFO:root:Training preset #8/1

            exp_gap   pbe_gap
mp-2520    0.802592  0.025146
mp-541407  0.647221  0.758261
mp-14025  -0.051152  0.032541
mp-5720   -0.041567  0.018705
mp-20581  -0.043653  0.020237
mae
0.44988406211999166


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7334633460> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.349
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.357
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.347
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.358
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.339
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.380
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.365
INFO:root:Training preset #8/1

           exp_gap   pbe_gap
mp-20715  0.012959  0.031253
mp-15986  0.015188  0.031430
mp-10133  0.012715  0.031485
mp-2418   1.419075 -0.299332
mp-2472   0.220758 -0.774587
mae
0.40030897566721296


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f735c2a29a0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.412
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.384
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.437
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.384
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.376
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.408
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.386
INFO:root:Training preset #8/1

            exp_gap   pbe_gap
mp-23222  -0.708200 -0.354188
mp-12780  -0.007077 -0.011916
mp-2556    0.007094 -0.012063
mp-1743   -0.005586 -0.014490
mp-866685 -0.007826 -0.010980
mae
0.3244034972128829


In [8]:
maes.mean()

0.4045033735837458

# exp alone: exact same train-test folds

In [10]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'train_jointA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        temp = copy.deecopy(train.df_targets)
        train.df_targets.drop('pbe_gap',axis=1,inplace=True)
        train.feature_selection(n=-1)
        train.df_targets = temp
        train.save(fpath)
    
    # assure no overlap
    print(len(set(train.df_targets.index).intersection(set(test.df_targets.index))))
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    #phase 1
    model = MODNetModel([[['exp_gap']]],{'exp_gap':1},act='elu')
    model.model.layers[-1].activation = tf.keras.activations.relu
    #print(model.model.summary())
    model.fit_preset(train,verbose=0)
    
    pred = model.predict(test)
    print(pred['exp_gap'].nsmallest(n=5))
    true = test.df_targets
    error = pred-true
    print(error.head())
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error['exp_gap'].values).mean()
    print('mae')
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f735c2a29a0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...


0


INFO:root:Validation loss: 0.412
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.425
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.389
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.429
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.403
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.406
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.393
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.358
INFO:root:Training preset #9/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.411

mp-8894    -0.206520
mp-10289   -0.054868
mp-30353   -0.024797
mp-20446   -0.022955
mp-10910   -0.022781
Name: exp_gap, dtype: float32
           exp_gap  pbe_gap
mp-8046  -0.013438      NaN
mp-12569  2.944123      NaN
mp-1211  -1.719579      NaN
mp-1723  -0.020542      NaN
mp-21149  0.009200      NaN
mae
0.4054699052092022
0


INFO:root:Validation loss: 0.348
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.330
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.387
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.368
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.340
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.364
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.375
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.316
INFO:root:Training preset #9/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.345

...

mp-4243    -0.009234
mp-14025   -0.009234
mp-5720    -0.009234
mp-20581   -0.009234
mp-30060   -0.009234
Name: exp_gap, dtype: float32
            exp_gap  pbe_gap
mp-2520    0.826543      NaN
mp-541407  0.445122      NaN
mp-14025  -0.009234      NaN
mp-5720   -0.009234      NaN
mp-20581  -0.009234      NaN
mae
0.41007692463170886
0


INFO:root:Validation loss: 0.355
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.380
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.393
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.369
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.358
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.385
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.375
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.352
INFO:root:Training preset #9/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.371

mp-30652     0.010638
mp-22017     0.010855
mp-4288      0.010856
mp-28795     0.010870
mp-573498    0.010881
Name: exp_gap, dtype: float32
           exp_gap  pbe_gap
mp-20715  0.010932      NaN
mp-15986  0.010932      NaN
mp-10133  0.010932      NaN
mp-2418   1.180664      NaN
mp-2472   0.177695      NaN
mae
0.41450707417526544
0


INFO:root:Validation loss: 0.453
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.458
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.389
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.402
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.373
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.410
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.377
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.367
INFO:root:Training preset #9/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.408

mp-583234   -0.021648
mp-9277     -0.014182
mp-10036    -0.012590
mp-24650    -0.012590
mp-1890     -0.012590
Name: exp_gap, dtype: float32
            exp_gap  pbe_gap
mp-23222  -0.712400      NaN
mp-12780  -0.012585      NaN
mp-2556    0.664100      NaN
mp-1743   -0.012590      NaN
mp-866685 -0.012589      NaN
mae
0.34134119151911535


In [11]:
maes.mean()

0.38120515328212

#### Conclusion

No improvement.